In [4]:
import tkinter as tk
from tkinter import messagebox
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from collections import Counter

meetings = []  # Store all meetings

def schedule_meeting():
    """Validates input, checks for conflicts, and schedules the meeting."""
    # Retrieve all entry values simultaneously
    inputs = [e.get().strip() for e in entries_list]
    title, date_str, time_str, duration_str, participants_str, venue_link_val = inputs
    participants = [p.strip() for p in participants_str.split(",") if p.strip()]
    mode = mode_var.get()

    # Normalize venue/link for comparison (strip + lower)
    new_venue_norm = venue_link_val.strip().lower()

    #  1. Basic Field and Participant Validation 
    if not all([title, date_str, time_str, duration_str, participants_str, venue_link_val, participants]):
        messagebox.showerror("Error", "All fields and participants are required!")
        return

    #  2. Date/Time/Duration Validation and Conversion 
    try:
        duration_minutes = int(duration_str)
        if not (1 <= duration_minutes <= 240): 
            raise ValueError
    except ValueError:
        messagebox.showerror("Error", "Duration must be 1–240 minutes.")
        return

    try:
        start_datetime = datetime.strptime(f"{date_str} {time_str}", "%Y-%m-%d %H:%M")
    except ValueError:
        messagebox.showerror("Error", "Invalid Date (YYYY-MM-DD) or Time (HH:MM) format.")
        return

    end_datetime = start_datetime + timedelta(minutes=duration_minutes)
    if start_datetime < datetime.now().replace(second=0, microsecond=0):
        messagebox.showerror("Error", "Meetings cannot be scheduled for a past date/time.")
        return

    #  3. Conflict Check (Participant and Venue) 
    for m in meetings:
        m_start = m["start_datetime"]
        m_end = m_start + timedelta(minutes=m["Duration"])
        time_overlap = (start_datetime < m_end) and (end_datetime > m_start)

        # A. Participant Conflict: if any participant is busy in overlapping time -> conflict
        participants_overlap = any(p in m["Participants"] for p in participants)
        if participants_overlap and time_overlap:
            messagebox.showwarning("Conflict", f"Participants busy with '{m['Title']}' at this time.")
            return

        # B. Venue Conflict (Offline meetings only): 
        # Normalize existing meeting venue and compare; if same venue and times overlap -> conflict
        # This check should block booking even if participants are different.
        existing_venue_norm = str(m.get("Venue/Link","")).strip().lower()
        if (mode == "Offline") and (m.get("Mode") == "Offline") and existing_venue_norm and new_venue_norm:
            if existing_venue_norm == new_venue_norm and time_overlap:
                messagebox.showwarning("Conflict", f"The venue '{venue_link_val}' is already booked for '{m['Title']}' at that time.")
                return

    #  4. Schedule Meeting & Update UI 
    meetings.append({
        "Title": title, "Date": date_str, "Time": time_str, "Duration": duration_minutes,
        "Participants": participants, "Mode": mode, "Venue/Link": venue_link_val,
        "start_datetime": start_datetime 
    })
    messagebox.showinfo("Success", "Meeting scheduled successfully!")
    update_listbox()
    clear_entries()

def clear_all_meetings():
    meetings.clear()
    update_listbox()
    messagebox.showinfo("Cleared", "All scheduled meetings have been cleared.")



def update_listbox():
    """Refreshes the Listbox with all scheduled meetings."""
    listbox.delete(0, tk.END)
    for m in sorted(meetings, key=lambda x: x["start_datetime"]):
        listbox.insert(tk.END, 
            f"[{m['Mode']}] {m['Date']} {m['Time']} ({m['Duration']}m) - {m['Title']} @ {m['Venue/Link']} | {', '.join(m['Participants'])}"
        )

def clear_entries():
    """Clears all input fields."""
    for entry in entries_list:
        entry.delete(0, tk.END)
    mode_var.set("Offline")
    toggle_venue_link()


def toggle_venue_link():
    """Update the label and required field behavior for venue/link based on mode."""
    if mode_var.get() == "Offline":
        l_venue_link.config(text="Venue:")
    else:
        l_venue_link.config(text="Meeting Link:")

#  Matplotlib plot popups with safe color generation 

def show_meetings_per_day():
    if not meetings:
        messagebox.showinfo("No data", "No meetings scheduled yet. Schedule meetings first.")
        return

    dates = [m['Date'] for m in meetings]
    counts = Counter(dates)
    x = sorted(counts.keys())
    y = [counts[d] for d in x]

    # use colormap to get as many colors as needed
    cmap = plt.get_cmap('tab20')
    colors = [cmap(i % cmap.N) for i in range(len(x))]

    popup = tk.Toplevel(root)
    popup.title("Meetings per Day")
    fig, ax = plt.subplots(figsize=(6,3.5))
    ax.bar(x, y, color=colors)
    ax.set_xlabel("Date")
    ax.set_ylabel("Number of Meetings")
    ax.set_title("Meetings per Day")
    plt.xticks(rotation=45)
    fig.tight_layout()
    canvas = FigureCanvasTkAgg(fig, master=popup)
    canvas.draw()
    canvas.get_tk_widget().pack(fill='both', expand=True)

def show_meetings_per_participant():
    if not meetings:
        messagebox.showinfo("No data", "No meetings scheduled yet. Schedule meetings first.")
        return

    # Flatten participant list and count
    all_parts = []
    for m in meetings:
        all_parts.extend(m['Participants'])
    counts = Counter(all_parts)
    if not counts:
        messagebox.showinfo("No participants", "No participants found in meetings.")
        return

    top_items = counts.most_common()
    names = [t[0] for t in top_items]
    values = [t[1] for t in top_items]

    # Use a matplotlib colormap to generate as many colors as needed
    cmap = plt.get_cmap('tab20')
    colors = [cmap(i % cmap.N) for i in range(len(names))]

    # Create popup
    popup = tk.Toplevel(root)
    popup.title("Meetings per Participant")
    fig, ax = plt.subplots(figsize=(6, max(3, 0.4*len(names))))
    ax.barh(names, values, color=colors)
    ax.set_xlabel("Number of Meetings")
    ax.set_title("Meetings per Participant")
    fig.tight_layout()
    canvas = FigureCanvasTkAgg(fig, master=popup)
    canvas.draw()
    canvas.get_tk_widget().pack(fill='both', expand=True)

def show_duration_distribution():
    """Histogram of meeting durations."""
    if not meetings:
        messagebox.showinfo("No data", "No meetings scheduled yet. Schedule meetings first.")
        return

    durations = [m['Duration'] for m in meetings]
    if not durations:
        messagebox.showinfo("No durations", "No duration data available.")
        return

    popup = tk.Toplevel(root)
    popup.title("Duration Distribution")
    fig, ax = plt.subplots(figsize=(6,3.5))
    bins = range(0, 241, 15)  # 15-minute bins
    ax.hist(durations, bins=bins, color='#FF69B4', edgecolor='black')
    ax.set_xlabel("Duration (minutes)")
    ax.set_ylabel("Count")
    ax.set_title("Distribution of Meeting Durations")
    fig.tight_layout()
    canvas = FigureCanvasTkAgg(fig, master=popup)
    canvas.draw()
    canvas.get_tk_widget().pack(fill='both', expand=True)

#                  Tkinter GUI Setup 
root = tk.Tk()
root.title("Advanced Meeting Scheduler")
root.configure(bg="#E8F0F2")
mode_var = tk.StringVar(value="Offline")

# Input fields
labels = ["Title:", "Date (YYYY-MM-DD):", "Time (HH:MM):", "Duration (min):", "Participants (comma-sep):"]
entries_list = []

for text in labels:
    tk.Label(root, text=text, bg="#E8F0F2").pack(pady=(5, 0), padx=10, anchor='w')
    entry = tk.Entry(root, width=50)
    entry.pack(pady=(0, 5), padx=10, fill='x')
    entries_list.append(entry)

e_title, e_date, e_time, e_duration, e_participants = entries_list

# Mode selection
tk.Label(root, text="Mode:", bg="#E8F0F2").pack(pady=(5, 0), padx=10, anchor='w')
radio_frame = tk.Frame(root, bg="#E8F0F2")
radio_frame.pack(pady=5, padx=10, fill='x')
tk.Radiobutton(radio_frame, text="Online", variable=mode_var, value="Online", bg="#E8F0F2", command=toggle_venue_link).pack(side=tk.LEFT, padx=5)
tk.Radiobutton(radio_frame, text="Offline", variable=mode_var, value="Offline", bg="#E8F0F2", command=toggle_venue_link).pack(side=tk.LEFT, padx=5)

# Venue/Link entry
l_venue_link = tk.Label(root, text="Venue:", bg="#E8F0F2")
l_venue_link.pack(pady=(5, 0), padx=10, anchor='w')
e_venue_link = tk.Entry(root, width=50)
e_venue_link.pack(pady=(0, 10), padx=10, fill='x')
entries_list.append(e_venue_link)

# Buttons with different colors 
btn_frame = tk.Frame(root, bg="#E8F0F2")
btn_frame.pack(padx=10, pady=8, fill='x')

tk.Button(btn_frame, text="Schedule Meeting", command=schedule_meeting, bg="#0000FF", fg="white", font=('Arial', 10, 'bold')).pack(side='left', padx=(0,8), ipadx=6)
tk.Button(btn_frame, text="Meetings per Day", command=show_meetings_per_day, bg="#4CAF50", fg="white").pack(side='left', padx=4, ipadx=6)
tk.Button(btn_frame, text="Meetings per Participant", command=show_meetings_per_participant, bg="#E74C3C", fg="white").pack(side='left', padx=4, ipadx=6)
tk.Button(btn_frame, text="Duration Distribution", command=show_duration_distribution, bg="#FF69B4", fg="white").pack(side='left', padx=4, ipadx=6)
tk.Button(btn_frame, text="Clear All Meetings", command=clear_all_meetings, bg="#B22222", fg="white").pack(side='left', padx=4, ipadx=6)

# Listbox
tk.Label(root, text="Scheduled Meetings:", bg="#E8F0F2", font=('Arial', 10, 'bold')).pack(pady=(5, 0), padx=10, anchor='w')
listbox = tk.Listbox(root, width=120, height=12)
listbox.pack(pady=10, padx=10, fill='both', expand=True)

toggle_venue_link()
root.mainloop()